In [1]:
# Calculate sBERT Embeddings

In [3]:
import sys

sys.version

'3.8.13 | packaged by conda-forge | (default, Mar 25 2022, 06:04:10) \n[GCC 10.3.0]'

In [4]:
def transform_to_sbert_embeddings(input):
    import os
    import json
    import sys
    # I created the venv-environment below using 
    
    #VENV_LIB_DIR = '/mnt/ceph/storage/data-in-progress/data-research/web-search/ECIR-22/ecir22-zero-shot/sbert-env/lib/python3.8/site-packages'
    #VENV_LIB_DIR = '/mnt/ceph/storage/data-in-progress/data-research/web-search/ECIR-22/ecir22-zero-shot/sbert-env-python3.7/lib/python3.7/site-packages'
    VENV_LIB_DIR = '/mnt/ceph/storage/data-in-progress/data-research/web-search/ECIR-22/ecir22-zero-shot/sbert-env-python3.8/lib/python3.8/site-packages'
    if VENV_LIB_DIR not in sys.path:
        sys.path = [VENV_LIB_DIR] + sys.path
    os.environ['SENTENCE_TRANSFORMERS_HOME'] = '/mnt/ceph/storage/data-in-progress/data-research/web-search/ECIR-22/ecir22-zero-shot/sbert-env/sentence-transformer-home'
    from sentence_transformers import SentenceTransformer
    sbert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    return json.dumps(sbert_model.encode(input).tolist())

In [5]:
transform_to_sbert_embeddings('hello world foo')

'[-0.2869361340999603, 0.21075160801410675, 0.17087240517139435, -0.21817827224731445, 0.30960220098495483, -0.4539799094200134, 0.6447420716285706, -0.42383041977882385, 0.057523518800735474, 0.29355257749557495, -0.056013356894254684, -0.9076987504959106, 0.13708749413490295, 0.14686094224452972, 0.666681706905365, -0.11197157204151154, 0.11379440128803253, -0.5456849336624146, -0.6463279724121094, -0.5853947401046753, -0.2384648621082306, 0.05953194573521614, 0.20145022869110107, 0.18753255903720856, 0.06930533051490784, 0.08370830863714218, -0.12975147366523743, 0.19339627027511597, 0.8819394111633301, 0.22021707892417908, 0.09086017310619354, 0.23316843807697296, 0.12768423557281494, 0.13054482638835907, 0.11542203277349472, 0.5505728125572205, 0.3658863306045532, 0.21320843696594238, -0.035716790705919266, -0.31554749608039856, -0.05744757503271103, -0.47509145736694336, 0.4187689423561096, 0.1606564223766327, -0.22457638382911682, -0.11338746547698975, -0.42795515060424805, 0.14

In [3]:
import pyspark

sc = pyspark.SparkContext()
sc

22/02/08 14:21:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/02/08 14:21:58 WARN Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.
22/02/08 14:21:59 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
22/02/08 14:22:14 WARN Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.


<SparkContext master=yarn appName=pyspark-shell>

In [4]:
def load_queries(file_name):
    ret = []
    import json
    from tqdm import tqdm
    with open(file_name, 'r') as f:
        for l in tqdm(f):
            ret += [json.loads(l)]
    return ret
    
queries = load_queries('/mnt/ceph/storage/data-in-progress/data-research/web-search/ECIR-22/ecir22-zero-shot/msmarco-document-train.jsonl')

367013it [00:05, 73188.42it/s]


In [5]:
queries[:3]

[{'id': '1185869',
  'contents': ')what was the immediate impact of the success of the manhattan project?',
  'target_document': ['D59219']},
 {'id': '1185868',
  'contents': '_________ justice is designed to repair the harm to victim, the community and the offender caused by the offender criminal act. question 19 options:',
  'target_document': ['D59235']},
 {'id': '1183785',
  'contents': 'elegxo meaning',
  'target_document': ['D576811']}]

In [5]:
def append_sbert_embedding(i):
    import json
    i['sbert-embedding'] = transform_to_sbert_embeddings(i['contents'])
    return json.dumps(i)

sc.parallelize(queries, 10000)\
    .map(append_sbert_embedding)\
    .saveAsTextFile(todo-find-directory-in-aql)

In [6]:
transform_to_sbert_embeddings('Hello World')

'[-0.08469964563846588, 0.45648717880249023, 0.1008683294057846, -0.11005671322345734, 0.04533912613987923, -0.36614954471588135, 0.35803312063217163, -0.0783013254404068, -0.24361993372440338, 0.27363526821136475, 0.1080302745103836, -0.7733523845672607, -0.1324445903301239, 0.19357404112815857, 0.20357096195220947, -0.3699706792831421, 0.33961641788482666, -0.6763169765472412, -0.849643886089325, -0.19371655583381653, -0.2306578904390335, -0.028220511972904205, 0.34298181533813477, 0.20185144245624542, 0.24069730937480927, -0.2726427912712097, 0.0705178901553154, 0.28514111042022705, 0.279055655002594, -0.010387003421783447, -0.011181145906448364, 0.04527720436453819, 0.24535025656223297, -0.020689554512500763, 0.047593019902706146, 0.4872283935546875, 0.11443371325731277, -0.12156311422586441, -0.153908833861351, -0.11076533794403076, 0.29678651690483093, -0.45546096563339233, 0.11479000747203827, 0.06491468846797943, -0.28057861328125, -0.005584833212196827, -0.03243649750947952, 0